In [2]:
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

# Load the data
df = pd.read_csv("CleanedData/player_stats_cleaned.csv")

# Initialize the Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='team-dropdown',
                options=[{'label': team, 'value': team} for team in df['team'].dropna().unique()],
                multi=True,
                placeholder="Select teams",
            ),
            dcc.Dropdown(
                id='player-dropdown',
                options=[],
                multi=True,
                placeholder="Select players",
            ),
        ], width=4),
        dbc.Col([
            dcc.Graph(id='scatter-plot'),
        ], width=8),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='bar-chart'), width=6),
        dbc.Col(dcc.Graph(id='box-plot'), width=6),
    ])
], fluid=True)

# Callback to update player options based on selected team
@app.callback(
    Output('player-dropdown', 'options'),
    [Input('team-dropdown', 'value')]
)
def update_player_dropdown(selected_teams):
    if selected_teams:
        filtered_df = df[df['team'].isin(selected_teams)]
    else:
        filtered_df = df
    return [{'label': player, 'value': player} for player in filtered_df['player'].unique()]

# Callback to update scatter plot and additional visualizations
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('bar-chart', 'figure'),
     Output('box-plot', 'figure')],
    [Input('team-dropdown', 'value'),
     Input('player-dropdown', 'value')]
)
def update_visualizations(selected_teams, selected_players):
    filtered_df = df

    if selected_teams:
        filtered_df = filtered_df[filtered_df['team'].isin(selected_teams)]
    if selected_players:
        filtered_df = filtered_df[filtered_df['player'].isin(selected_players)]

    # Scatter plot for goals vs assists
    scatter_fig = px.scatter(
        filtered_df,
        x='goals',
        y='assists',
        color='team',
        hover_data=['player'],
        title='Goals vs Assists by Player'
    )

    # Bar chart for games played
    bar_fig = px.bar(
        filtered_df,
        x='player',
        y='games',
        color='team',
        title='Games Played by Player'
    )

    # Box plot for aerials won
    box_fig = px.box(
        filtered_df,
        y='aerials_won',
        color='team',
        title='Aerials Won Distribution by Team'
    )

    return scatter_fig, bar_fig, box_fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8053)
